<a href="https://colab.research.google.com/github/mishras/nexmind/blob/main/sprint_slipping_feature_delay_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import logging
from jira import JIRA
from git import Repo
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Jira Configuration
JIRA_SERVER = os.getenv('JIRA_SERVER')
JIRA_USERNAME = os.getenv('JIRA_USERNAME')
JIRA_TOKEN = os.getenv('JIRA_TOKEN')
JIRA_PROJECT_KEY = os.getenv('JIRA_PROJECT_KEY')

# Git Configuration
GIT_REPO_PATH = os.getenv('GIT_REPO_PATH')
GIT_BRANCH_NAME = os.getenv('GIT_BRANCH_NAME', 'main')

# Define a function to connect to Jira
def connect_to_jira():
    try:
        options = {'server': JIRA_SERVER}
        jira = JIRA(options, basic_auth=(JIRA_USERNAME, JIRA_TOKEN))
        logging.info("Connected to Jira successfully.")
        return jira
    except Exception as e:
        logging.error(f"Failed to connect to Jira: {e}")
        raise

# Define a function to fetch Jira issues
def get_jira_issues(jira):
    try:
        jql = f"project = {JIRA_PROJECT_KEY} AND status != Done"
        issues = jira.search_issues(jql, maxResults=1000)
        logging.info(f"Fetched {len(issues)} Jira issues.")
        return issues
    except Exception as e:
        logging.error(f"Failed to fetch Jira issues: {e}")
        raise

# Define a function to load git data
def load_git_data(repo_path, branch_name, lookback_days=30):
    try:
        repo = Repo(repo_path)
        commits = list(repo.iter_commits(branch_name, since=(datetime.now() - timedelta(days=lookback_days))))
        logging.info(f"Loaded {len(commits)} commits from branch {branch_name}.")

        commit_data = {}
        for commit in commits:
            message = commit.message.strip()
            # Assuming Jira ticket keys are included in the commit messages
            ticket_keys = [part for part in message.split() if part.startswith(JIRA_PROJECT_KEY)]
            for key in ticket_keys:
                if key in commit_data:
                    commit_data[key] += 1
                else:
                    commit_data[key] = 1

        return commit_data
    except Exception as e:
        logging.error(f"Failed to load git data: {e}")
        raise

# Define a function to analyze issues for red flags
def analyze_issues(issues, git_data):
    red_flags = []

    for issue in issues:
        key = issue.key
        story_points = issue.fields.customfield_10004  # Replace with your story point field ID
        commits = git_data.get(key, 0)

        # 1. No Git Commits
        if commits == 0:
            red_flags.append((key, "No commits made"))

        # 2. No Technical Design Documentation
        if not issue.fields.customfield_10005:  # Replace with your technical design field ID
            red_flags.append((key, "No technical design documentation"))

        # 3. No Meetings Related to Jira Ticket
        if issue.fields.timespent == 0:  # Assuming 'timespent' field tracks meeting time
            red_flags.append((key, "No meetings related to Jira ticket"))

        # 4. No Stand-up Updates
        if not issue.fields.customfield_10006:  # Replace with your standup update field ID
            red_flags.append((key, "No stand-up updates"))

        # 5. Lack of Code Reviews / Merge Requests
        if 'review_comments' not in issue.fields or issue.fields.review_comments == 0:
            red_flags.append((key, "No code reviews or merge requests"))

        # 6. No Test Cases or Test Plans
        if not issue.fields.customfield_10007:  # Replace with your test cases field ID
            red_flags.append((key, "No test cases or test plans"))

        # 7. Insufficient Branch Activity
        if commits < (story_points or 1):
            red_flags.append((key, "Insufficient branch activity"))

        # 8. Low Code Coverage
        # This would typically come from a CI tool integrated with Jira; for now, assume placeholder logic
        if 'code_coverage' in issue.fields and issue.fields.code_coverage < 80:
            red_flags.append((key, "Low code coverage"))

        # 9. Stale Dependencies
        if 'dependency_age' in issue.fields and issue.fields.dependency_age > 60:  # Example: dependency not updated for 60 days
            red_flags.append((key, "Stale dependencies"))

        # 10. Blocked Issues
        if issue.fields.status.name == "Blocked":
            red_flags.append((key, "Issue is blocked"))

        # 11. Cross-team Dependencies Not Addressed
        if 'cross_team_dependencies' in issue.fields and not issue.fields.cross_team_dependencies:
            red_flags.append((key, "Cross-team dependencies not addressed"))

        # 12. Inadequate Task Breakdown
        if story_points > 8 and len(issue.fields.subtasks) < 3:
            red_flags.append((key, "Inadequate task breakdown"))

        # 13. High PR-to-Merge Ratio
        if 'pull_requests' in issue.fields and issue.fields.pull_requests > 5 and commits < 5:
            red_flags.append((key, "High PR-to-merge ratio, potential bottleneck"))

    return red_flags

# Main execution function
def main():
    try:
        # Step 1: Connect to Jira
        jira = connect_to_jira()

        # Step 2: Fetch Jira issues
        issues = get_jira_issues(jira)

        # Step 3: Load git data
        git_data = load_git_data(GIT_REPO_PATH, GIT_BRANCH_NAME)

        # Step 4: Analyze issues for red flags
        red_flags = analyze_issues(issues, git_data)

        # Step 5: Display or log the results
        if red_flags:
            for flag in red_flags:
                logging.warning(f"Issue {flag[0]}: {flag[1]}")
        else:
            logging.info("No red flags detected.")

    except Exception as e:
        logging.error(f"Script failed: {e}")
        raise

if __name__ == "__main__":
    main()
